<a href="https://colab.research.google.com/github/mrarat76/MachineLearningProjects/blob/main/whisper_arat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Gerekli kütüphaneleri kur
!pip install git+https://github.com/openai/whisper.git
!pip install yt-dlp
!pip install ffmpeg-python
!apt-get install ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-qiidqplu
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-qiidqplu
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-

In [2]:
import whisper
import yt_dlp
import ffmpeg
import os

In [3]:
# Videoyu indir
def download_video(url, output_format='mp4'):
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': f'input.{output_format}',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])


In [4]:
# Ses dosyasını farklı formatlara dönüştür
def convert_audio(input_file, output_format='mp3'):
    output_file = f'input.{output_format}'
    ffmpeg.input(input_file).output(output_file).run(overwrite_output=True)
    return output_file

In [5]:
# Whisper modelini kullanarak transkripsiyon yap
def transcribe_audio(model, audio_file):
    result = model.transcribe(audio_file)
    return result['text'], result['segments']


In [6]:
# Transkripsiyon sonuçlarını kaydet
def save_transcription(text, segments, text_file='output.txt', srt_file='output.srt'):
    with open(text_file, 'w') as f:
        f.write(text)

    with open(srt_file, 'w') as f:
        for i, segment in enumerate(segments):
            start = segment['start']
            end = segment['end']
            text = segment['text']
            f.write(f"{i+1}\n")
            f.write(f"{format_time(start)} --> {format_time(end)}\n")
            f.write(f"{text}\n\n")

In [7]:
def format_time(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = int((seconds - int(seconds)) * 1000)
    return f"{int(hours):02}:{int(minutes):02}:{int(seconds):02},{milliseconds:03}"


In [8]:
def main():
    url = "https://www.youtube.com/watch?v=mgpPwEHx8Mw"  # URL'i güncelleyin
    download_video(url)

    mp4_file = 'input.mp4.webm'  # İndirilen dosya ismi
    mp3_file = convert_audio(mp4_file, 'mp3')
    wav_file = convert_audio(mp4_file, 'wav')

    model = whisper.load_model("base")

    text, segments = transcribe_audio(model, mp3_file)
    save_transcription(text, segments, 'output_mp3.txt', 'output_mp3.srt')

    text, segments = transcribe_audio(model, wav_file)
    save_transcription(text, segments, 'output_wav.txt', 'output_wav.srt')

if __name__ == "__main__":
    main()

[youtube] Extracting URL: https://www.youtube.com/watch?v=mgpPwEHx8Mw
[youtube] mgpPwEHx8Mw: Downloading webpage
[youtube] mgpPwEHx8Mw: Downloading ios player API JSON
[youtube] mgpPwEHx8Mw: Downloading player 71059424
[youtube] mgpPwEHx8Mw: Downloading m3u8 information
[info] mgpPwEHx8Mw: Downloading 1 format(s): 248+251
[download] Destination: input.mp4.f248.webm
[download] 100% of   47.23MiB in 00:00:23 at 2.05MiB/s   
[download] Destination: input.mp4.f251.webm
[download] 100% of    4.33MiB in 00:00:02 at 2.10MiB/s   
[Merger] Merging formats into "input.mp4.webm"
Deleting original file input.mp4.f251.webm (pass -k to keep)
Deleting original file input.mp4.f248.webm (pass -k to keep)


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 111MiB/s]
